In [1]:
import tensorflow as tf
import sys
sys.path.append("../transformer/")
from tqdm import tqdm
from data_load import get_batch_data, load_de_vocab, load_en_vocab
from nltk.translate.bleu_score import corpus_bleu
from data_load import load_test_data 
import os
import codecs
import numpy as np

/home/hadoop/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class Hyperparams(object):
    #data
    source_train = 'corpora/train.tags.de-en.de'
    target_train = 'corpora/train.tags.de-en.en'
    source_test = 'corpora/IWSLT16.TED.tst2014.de-en.de.xml'
    target_test = 'corpora/IWSLT16.TED.tst2014.de-en.en.xml'
    
    # training
    batch_size=32
    lr = 0.001
    logdir = 'logdir' # log directory
    #model
    maxlen = 10 # Maximum number of words in a sentence. alias = T.
                # Feel free to increase this if you are ambitious.
    min_cnt = 20 # words whose occurred less than min_cnt are encoded as <UNK>.
    hidden_units = 512 # alias = C
    num_blocks = 6 # number of encoder/decoder blocks
    num_epochs = 1
    num_heads = 8
    units_per_head=1024
    dropout_rate = 0.1
    is_training=True
    num_layers=6
hp=Hyperparams()

In [3]:
def multihead_attention(query,key,value,seq_len,units_per_head=1024,num_heads=8,mask=False,scope="multihead_attention", 
reuse=None):
    #使用时一般K和V一样
    #query(N,Tq/h,C) key=value(N,Tk/h,C)
    with tf.variable_scope(scope, reuse=reuse):
#         units_per_head = query.shape[-1]
        
        num_units = units_per_head
        
        #做线性映射，直接um_units*num_heads宽度与num_heads个um_units线性映射是一样的
        Q=tf.layers.dense(query,num_units*num_heads,activation=tf.nn.relu)# (N, T_q, C)
        K=tf.layers.dense(key,num_units*num_heads,activation=tf.nn.relu)# (N, T_k, C)
        V=tf.layers.dense(value,num_units*num_heads,activation=tf.nn.relu)# (N, T_k, C)
        #tf.split(dimension, num_split, input)：dimension的意思就是输入张量的哪一个维度，
        #如果是0就表示对第0维度进行切割。num_split就是切割的数量
        Q=tf.concat(tf.split(Q,num_heads,axis=2),axis=0)#(h*N,Tq,C/h)
        K=tf.concat(tf.split(K,num_heads,axis=2),axis=0)#(h*N,Tk,C/h)
        V=tf.concat(tf.split(V,num_heads,axis=2),axis=0)#(h*N,Tk,C/h)
        #计算内积，然后mask，然后softmax
        A = tf.matmul(Q,tf.transpose(K,[0,2,1]))#(h*N,Tq,Tk)
        A = A /(K.get_shape().as_list()[-1] ** 0.5)

        # Key Masking
        #将元素中最后一个为度之和全为0的位置标记为0 source(N, T_k,word_dim)
        key_masks = tf.sign(tf.abs(tf.reduce_sum(key, axis=-1))) # (N, T_k)
        key_masks = tf.tile(key_masks, [num_heads, 1]) # (h*N, T_k)
        #由于每个queries都要对应这些keys，而mask的key对每个queries都是mask的
        key_masks = tf.tile(tf.expand_dims(key_masks, 1), [1, Q.shape[1], 1]) # (h*N, T_q, T_k)
        #定义一个和outputs同shape的paddings，该tensor每个值都设定的极小。用where函数比较，当对应位置的key_masks值为0
        #也就是需要mask时，outputs的该值（attention score）设置为极小的值（利用paddings实现），否则保留原来的outputs值。
        paddings = tf.ones_like(A)*(-2**32+1)
        A = tf.where(tf.equal(key_masks, 0), paddings, A) # (h*N, T_q, T_k)

        #Causality Future blinding
        if mask:
            #首先定义一个和outputs后两维的shape相同shape（T_q,T_k）的一个张量（矩阵）。
            #然后将该矩阵转为三角阵tril。三角阵中，对于每一个T_q,凡是那些大于它角标的T_k值全都为0，
            #这样作为mask就可以让query只取它之前的key（self attention中query即key）。
            #由于该规律适用于所有query，接下来仍用tile扩展堆叠其第一个维度，构成masks，shape为(h*N, T_q,T_k).
            diag_vals = tf.ones_like(A[0, :, :]) # (T_q, T_k)
            tril = tf.contrib.linalg.LinearOperatorTriL(diag_vals).to_dense() # (T_q, T_k)
            masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(A)[0], 1, 1]) # (h*N, T_q, T_k)
            paddings = tf.ones_like(masks)*(-2**32+1)
            A = tf.where(tf.equal(masks, 0), paddings, A) # (h*N, T_q, T_k)
        A = tf.nn.softmax(A)# (h*N, T_q, T_k)

        # Query Masking
        query_masks = tf.sign(tf.abs(tf.reduce_sum(query, axis=-1))) # (N, T_q)
        query_masks = tf.tile(query_masks, [num_heads, 1]) # (h*N, T_q)
        query_masks = tf.tile(tf.expand_dims(query_masks, -1), [1, 1, tf.shape(key)[1]]) # (h*N, T_q, T_k)

        A *= query_masks # broadcasting. (N, T_q, C)
        A=tf.layers.dropout(A, rate=hp.dropout_rate)

        # weighted sum
        outputs=tf.matmul(A,V)# ( h*N, T_q, C/h)

        #Restore shape
        outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2 ) # (N, T_q, C)
        #将形状变回原来形状
        outputs = tf.layers.dense(outputs,query.shape[-1])
        # Residual connection
        outputs += query

        #Normalize
        outputs=layer_normal(outputs)
    return outputs

def feedforward(inputs,num_units=[2048,512],reuse=None):
    outputs= tf.layers.conv1d(
        inputs,filters=num_units[0],kernel_size=1,activation=tf.nn.relu,use_bias=True)
    outputs=tf.layers.conv1d(
        outputs,filters=num_units[1],kernel_size=1,activation=None,use_bias=True)
    # Residual connection
    outputs += inputs
    outputs=layer_normal(outputs)
    return outputs
def layer_normal(inputs,epsilon = 1e-8):
    mean,variance = tf.nn.moments(inputs,[-1],keep_dims=True)
    params_shape = inputs.shape[-1:]
    beta= tf.Variable(tf.zeros(params_shape))
    gamma = tf.Variable(tf.ones(params_shape))
    normalized = (inputs-mean)/(((variance + epsilon) ** (.5)))
    outputs = gamma * normalized + beta
    return outputs
def encoder (pre_enc,num_layers,hparams,scope,seq_len):
    for i in range(num_layers):
        scope="encoder_layer_{}".format(i)
        with tf.variable_scope(scope):
            outputs = multihead_attention(
                query=pre_enc,
                key=pre_enc,
                value=pre_enc,
                units_per_head=hparams.units_per_head,
                num_heads=hparams.num_heads,
                mask=False,
                seq_len=seq_len)
            outputs = feedforward(outputs, num_units=[4*hparams.hidden_units, hparams.hidden_units])
    return outputs
def decoder (enc,pre_dec,num_layers,hparams,scope,seq_len):
    for i in range(num_layers):
        scope="decoder_layer_{}".format(i)
        with tf.variable_scope(scope):
            output=multihead_attention(
                query=pre_dec,
                key=pre_dec,
                value=pre_dec,
                units_per_head=hparams.units_per_head,
                num_heads=hparams.num_heads,
                mask=True,
                scope="self_attention",
                seq_len=seq_len)
            output=multihead_attention(
                query=pre_dec,
                key=output,
                value=output,
                units_per_head=hparams.units_per_head,
                num_heads=hparams.num_heads,
                mask=False,
                scope="vanilla_attention",
                seq_len=seq_len)
            output = feedforward(output, num_units=[4*hp.hidden_units, hp.hidden_units])
    return output
def position_embedding(inputs,position_size):
    batch_size,seq_len = tf.shape(inputs)[0],tf.shape(inputs)[1]
    pos_j = 1. / tf.pow(10000., 2 * tf.range(position_size / 2, dtype=tf.float32) / position_size)
    pos_j = tf.expand_dims(pos_j, 0)
    pos_i = tf.range(tf.cast(seq_len, tf.float32), dtype=tf.float32)
    pos_i = tf.expand_dims(pos_i, 1)
    pos_ij = tf.matmul(pos_i, pos_j)
    pos_ij = tf.concat([tf.cos(pos_ij), tf.sin(pos_ij)], 1)
    position_embedding = tf.expand_dims(pos_ij, 0) + tf.zeros((batch_size, seq_len, position_size))
    return position_embedding
def word_embedding(inputs,vocab_size,num_units,zero_pad=True,scale=True,scope="embedding",reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        lookup_table = tf.get_variable('lookup_table',
                                       dtype=tf.float32,
                                       shape=[vocab_size, num_units],
                                       initializer=tf.contrib.layers.xavier_initializer())
        if zero_pad:
            lookup_table = tf.concat((tf.zeros(shape=[1, num_units]),lookup_table[1:, :]), 0)
        outputs = tf.nn.embedding_lookup(lookup_table, inputs)
        if scale:
            outputs = outputs * (num_units ** 0.5) 
    return outputs

def label_smoothing(inputs,epsilon=0.1):
    """Applies label smoothing. See https://arxiv.org/abs/1512.00567."""
    K = tf.to_float(tf.shape(inputs)[-1]) # number of channels
    return ((1.-epsilon)*inputs+(epsilon/K))
#     return ((1-epsilon)*inputs+(epsilon/K))

# # https://github.com/vahidk/EffectiveTensorflow#beam_search
# def beamDecoder(enc,dec,hp.num_layers,hp,seq_len,scope="beam_decoder"):
#     enc_output=tf.tile(tf.expand_dims(enc, axis=1),multiples=[1, hp.beam_width, 1])
#     seq_len=tf.tile(tf.expand_dims(enc_output, axis=1),multiples=[1, hp.beam_width, 1])
#     ids = tf.tile([[2]], [hp.batch_size, hp.beam_width])
#     sel_ids = tf.zeros([hp.batch_size, hp.beam_width, 0], dtype=ids.dtype)
#     mask = tf.ones([batch_size, beam_width], dtype=tf.float32)
#     for i in range(hp.maxlen):
#         with tf.variable_scope(scope,reuse=True if i>1 else None):
#             state, logits = update_fn(state, ids)
#             logits = tf.nn.log_softmax(logits)
#             sum_logprobs = (tf.expand_dims(sel_sum_logprobs, axis=2) + (logits * tf.expand_dims(mask, axis=2)))
#             num_classes = logits.shape.as_list()[-1]
#             sel_sum_logprobs, indices = tf.nn.top_k(
#                   tf.reshape(sum_logprobs, [batch_size, num_classes * beam_width]),
#                   k=beam_width)
#             ids = indices % num_classes
#             beam_ids = indices // num_classes
#             state = batch_gather(state, beam_ids)
#             sel_ids = tf.concat([batch_gather(sel_ids, beam_ids),
#                        tf.expand_dims(ids, axis=2)], axis=2)
#             mask = (batch_gather(mask, beam_ids) *
#                     tf.to_float(tf.not_equal(ids, end_token_id)))
#     return sel_ids, sel_sum_logprobs    
    

In [5]:
class Hyperparams(object):
    #data
    source_train = 'corpora/train.tags.de-en.de'
    target_train = 'corpora/train.tags.de-en.en'
    source_test = 'corpora/IWSLT16.TED.tst2014.de-en.de.xml'
    target_test = 'corpora/IWSLT16.TED.tst2014.de-en.en.xml'
    
    # training
    batch_size=2
    lr = 0.001
    logdir = 'logdir' # log directory
    #model
    maxlen = 2 # Maximum number of words in a sentence. alias = T.
                # Feel free to increase this if you are ambitious.
    min_cnt = 20 # words whose occurred less than min_cnt are encoded as <UNK>.
    hidden_units = 3 #512 alias = C
    num_blocks = 6 # number of encoder/decoder blocks
    num_epochs = 1
    num_heads = 1
    units_per_head=1024
    dropout_rate = 0.1
    is_training=True
    num_layers=1
    beam_width=2
hp=Hyperparams()
import tensorflow as tf
enc=tf.Variable([[[1.,1.,1.],[2.,2.,2.],[3.,3.,3.]],[[3.,3.,3.],[4.,4.,4.],[3.,3.,3.]]])
dec=tf.Variable([[[0.,0.,0.],[0.,0.,0.],[3.,3.,3.]],[[1.,0.,0.],[0.,0.,0.],[3.,3.,3.]]])
seq_len =tf.Variable( [[2],[2]])
# https://github.com/vahidk/EffectiveTensorflow#beam_search
def get_shape(tensor):
    """Returns static shape if available and dynamic shape otherwise."""
    static_shape = tensor.shape.as_list()
    dynamic_shape = tf.unstack(tf.shape(tensor))
    dims = [s[1] if s[0] is None else s[0]
          for s in zip(static_shape, dynamic_shape)]
    return dims
def batch_gather(tensor, indices):
    """Gather in batch from a tensor of arbitrary size.

    In pseudocode this module will produce the following:
    output[i] = tf.gather(tensor[i], indices[i])

    Args:
    tensor: Tensor of arbitrary size.
    indices: Vector of indices.
    Returns:
    output: A tensor of gathered values.
    """
    shape = get_shape(tensor)
    flat_first = tf.reshape(tensor, [shape[0] * shape[1]] + shape[2:])
    indices = tf.convert_to_tensor(indices)
    offset_shape = [shape[0]] + [1] * (indices.shape.ndims - 1)
    offset = tf.reshape(tf.range(shape[0]) * shape[1], offset_shape)
    output = tf.gather(flat_first, indices + offset)
    return output
def update_fn(state, ids):
    state = decoder (state,pre_dec,num_layers,hparams,scope,seq_len):
def beamDecoder(enc,dec_seq,num_layers,hp,seq_len):
    enc = tf.contrib.seq2seq.tile_batch(enc,hp.beam_width)
    dec_seq = tf.contribi.seq2seq.tile_batch(dec_seq,hp.beam_width)
    
    dec = word_embedding(dec_seq, vocab_size=len(en2idx), 
                              num_units=hp.hidden_units,scale=True,scope="dec_embed")
    dec += position_embedding(self.x,hp.hidden_units)
    begin_token_id=2
    end_token_id=1

    ids = tf.tile([[begin_token_id]], [batch_size, beam_width])
    sel_ids = tf.zeros([batch_size, beam_width, 0], dtype=ids.dtype)
    mask = tf.ones([batch_size, beam_width], dtype=tf.float32)
    
#     paths_ids = 
    for i in range(hp.maxlen):
        with tf.variable_scope(name, reuse=True if i > 0 else None):
            dec = decoder(enc,dec,num_layers,hparams,scope,seq_len):
    return sel_ids, sel_sum_logprobs
x=tf.Variable([[1.]+[0.]*9])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#     print(tf.log(x).eval())
    a,b=beamDecoder(enc,dec,hp.num_layers,hp,seq_len,scope="beam_decoder")
    print(a.eval())
    print(b.eval())

Tensor("Tile_2:0", shape=(3, 2, 3), dtype=float32)
Tensor("Tile_3:0", shape=(2, 2), dtype=int32)


ValueError: Variable beam_decoder/decoder_layer_0/self_attention/dense/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-3-2da132dbccdc>", line 11, in multihead_attention
    Q=tf.layers.dense(query,num_units*num_heads,activation=tf.nn.relu)# (N, T_q, C)
  File "<ipython-input-3-2da132dbccdc>", line 112, in decoder
    seq_len=seq_len)
  File "<ipython-input-4-457eb8131b02>", line 58, in update_fn
    state = decoder(state,ids,hp.num_layers,hp,"decoder",hp.maxlen)


In [5]:
class transformer(object):
    def __init__(self):
        self.graph = tf.Graph()
        with self.graph.as_default():
            if hp.is_training:
                self.x,self.y,self.num_batch = get_batch_data() # (N, T)
            else: #inference
                self.x = tf.placeholder(tf.int32, shape=(None, hp.maxlen))
                self.y = tf.placeholder(tf.int32, shape=(None, hp.maxlen))
                
            de2idx, idx2de = load_de_vocab()
            en2idx, idx2en = load_en_vocab()
            #[N,T,hp.hidden_units]
            self.enc = word_embedding(self.x,vocab_size=len(de2idx),num_units=hp.hidden_units,scale=True,scope="enc_embed")
            #[N,T,hp.hidden_units]??
            self.enc+= position_embedding(self.x,hp.hidden_units)
            self.enc = tf.layers.dropout(self.enc, rate=hp.dropout_rate,training=tf.convert_to_tensor(hp.is_training))
            seq_len=hp.maxlen
            self.enc = encoder(self.enc,hp.num_layers,hp,"encoder",seq_len)
            
            self.decoder_inputs = tf.concat((tf.ones_like(self.y[:, :1])*2, self.y[:, :-1]), -1) # 在开头添加2:<S>
            
                   
            if hp.is_training:
                self.dec = word_embedding(self.decoder_inputs, vocab_size=len(en2idx), 
                                          num_units=hp.hidden_units,scale=True,scope="dec_embed")
                self.dec += position_embedding(self.x,hp.hidden_units)
                self.dec = tf.layers.dropout(self.dec, rate=hp.dropout_rate,training=tf.convert_to_tensor(hp.is_training))
                self.dec = decoder(self.enc,self.dec,hp.num_layers,hp,"decoder",seq_len)
                
                # Final linear projection
                self.logits = tf.layers.dense(self.dec, len(en2idx)) #[N,T,len(en2idx)]
                self.preds = tf.to_int32(tf.argmax(self.logits, axis=-1),name="preds") #[N,T]
            elif hp.beam_width>0:
                self.logits = beamDecoder(self.enc,self.decoder_inputs,hp.num_layers,hp,seq_len,scope="beam_decoder")
#                 self.logits = tf.layers.dense(self.dec, len(en2idx)) #[N,T,len(en2idx)]
                self.preds = tf.to_int32(tf.argmax(self.logits, axis=-1),name="preds") #[N,T]
                
            if hp.is_training:
                #同时把label（即self.y）中所有id不为0（即是真实的word，不是pad）的位置的值用float型的1.0代替作为self.istarget，其shape为[N,T]
                self.istarget = tf.to_float(tf.not_equal(self.y, 0))
                #当self.preds和self.y中对应位置值相等时转为float 1.0,否则为0
                self.acc = tf.reduce_sum(tf.to_float(tf.equal(self.preds, self.y))*self.istarget)/ (tf.reduce_sum(self.istarget))

                #Loss
                self.y_smoothed = label_smoothing(tf.cast(tf.one_hot(self.y,depth=len(en2idx)),dtype=tf.float32))
                self.loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,labels=self.y_smoothed)
                self.mean_loss=tf.reduce_sum(self.loss*self.istarget)/(tf.reduce_sum(self.istarget))

                #Training Scheme
                self.global_step=tf.Variable(0,name='global_step',trainable=False)
                self.optimizer = tf.train.AdamOptimizer(learning_rate=hp.lr,beta1=0.9,beta2=0.98,epsilon=1e-8,name='optimizer')
                self.train_op= self.optimizer.minimize(self.loss,global_step=self.global_step,name='train_op')

                #Summary
                tf.summary.scalar('mean_loss',self.mean_loss)
                self.merged = tf.summary.merge_all()
   

In [7]:
class Beam(object):
    ''' Store the neccesary info for beam search. '''

    def __init__(self, size, cuda=False):

        self.size = size
        self.done = False

        self.tt = torch.cuda if cuda else torch

        # The score for each translation on the beam.
        self.scores = self.tt.FloatTensor(size).zero_()
        self.all_scores = []

        # The backpointers at each time-step.
        self.prev_ks = []

        # The outputs at each time-step.
        self.next_ys = [self.tt.LongTensor(size).fill_(Constants.PAD)]
        self.next_ys[0][0] = Constants.BOS

    def get_current_state(self):
        "Get the outputs for the current timestep."
        return self.get_tentative_hypothesis()

    def get_current_origin(self):
        "Get the backpointers for the current timestep."
        return self.prev_ks[-1]

    def advance(self, word_lk):
        "Update the status and check for finished or not."
        num_words = word_lk.size(1)

        # Sum the previous scores.
        if len(self.prev_ks) > 0:
            beam_lk = word_lk + self.scores.unsqueeze(1).expand_as(word_lk)
        else:
            beam_lk = word_lk[0]

        flat_beam_lk = beam_lk.view(-1)

        best_scores, best_scores_id = flat_beam_lk.topk(self.size, 0, True, True) # 1st sort
        best_scores, best_scores_id = flat_beam_lk.topk(self.size, 0, True, True) # 2nd sort

        self.all_scores.append(self.scores)
        self.scores = best_scores

        # bestScoresId is flattened beam x word array, so calculate which
        # word and beam each score came from
        prev_k = best_scores_id / num_words
        self.prev_ks.append(prev_k)
        self.next_ys.append(best_scores_id - prev_k * num_words)

        # End condition is when top-of-beam is EOS.
        if self.next_ys[-1][0] == Constants.EOS:
            self.done = True
            self.all_scores.append(self.scores)

        return self.done

    def sort_scores(self):
        "Sort the scores."
        return torch.sort(self.scores, 0, True)

    def get_the_best_score_and_idx(self):
        "Get the score of the best in the beam."
        scores, ids = self.sort_scores()
        return scores[1], ids[1]

    def get_tentative_hypothesis(self):
        "Get the decoded sequence for the current timestep."
        if len(self.next_ys) == 1:
            dec_seq = self.next_ys[0].unsqueeze(1)
        else:
            _, keys = self.sort_scores()
            hyps = [self.get_hypothesis(k) for k in keys]
            hyps = [[Constants.BOS] + h for h in hyps]
            dec_seq = torch.from_numpy(np.array(hyps))
        return dec_seq

    def get_hypothesis(self, k):
        """
        Walk back to construct the full hypothesis.
        Parameters.
             * `k` - the position in the beam to construct.
         Returns.

            1. The hypothesis
            2. The attention at each time step.
        """
        hyp = []
        for j in range(len(self.prev_ks) - 1, -1, -1):
            hyp.append(self.next_ys[j+1][k])
            k = self.prev_ks[j][k]
        return hyp[::-1]

In [8]:
def train():
    # Load vocabulary    
#     de2idx, idx2de = load_de_vocab()
#     en2idx, idx2en = load_en_vocab()
    model=transformer()
    sv=tf.train.Supervisor(graph=model.graph,logdir=hp.logdir,save_model_secs=0)
    with sv.managed_session() as sess:
        for epoch in range(1, hp.num_epochs+1):
            if sv.should_stop(): break
            for step in tqdm(range(hp.batch_size), total=hp.batch_size, ncols=70, leave=False, unit='b'):
                sess.run(model.train_op)
            gs = sess.run(model.global_step)
            sv.saver.save(sess, hp.logdir + '/model_epoch_%02d_gs_%d' % (epoch, gs))
            print("save done.")

In [9]:
train()
# eval()

<tf.Variable 'Variable:0' shape=(2, 3, 3) dtype=int32_ref>
INFO:tensorflow:Restoring parameters from logdir/model_epoch_01_gs_1


KeyboardInterrupt: 

In [ ]:
def eval(): 
    def translate_batch(model,src_batch):
        model.
    # Load graph
    hp.is_training=False
    model=transformer()
    print("Graph loaded")
    
    # Load data
    X, Sources, Targets = load_test_data()
    de2idx, idx2de = load_de_vocab()
    en2idx, idx2en = load_en_vocab()
     
#     X, Sources, Targets = X[:33], Sources[:33], Targets[:33]
     
    # Start session 
    with model.graph.as_default():    
        sv = tf.train.Supervisor()
        with sv.managed_session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
            ## Restore parameters
            sv.saver.restore(sess, tf.train.latest_checkpoint(hp.logdir))
            print("Restored!")
              
            ## Get model name
            mname = open(hp.logdir + '/checkpoint', 'r').read().split('"')[1] # model name
             
            ## Inference
            if not os.path.exists('results'): os.mkdir('results')
            with codecs.open("results/" + mname, "w", "utf-8") as fout:
                list_of_refs, hypotheses = [], []
                for i in range(len(X) // hp.batch_size):
                    ### Get mini-batches
                    x = X[i*hp.batch_size: (i+1)*hp.batch_size]
                    sources = Sources[i*hp.batch_size: (i+1)*hp.batch_size]
                    targets = Targets[i*hp.batch_size: (i+1)*hp.batch_size]
                    ### Autoregressive inference
                    preds = np.zeros((hp.batch_size, hp.maxlen), np.int32)
                    for j in range(hp.maxlen):
                        _preds = sess.run(model.preds, {model.x: x, model.y: preds})
                        preds[:, j] = _preds[:, j]
                     
                    ### Write to file
                    for source, target, pred in zip(sources, targets, preds): # sentence-wise
                        got = " ".join(idx2en[idx] for idx in pred).split("</S>")[0].strip()
                        fout.write("- source: " + source +"\n")
                        fout.write("- expected: " + target + "\n")
                        fout.write("- got: " + got + "\n\n")
                        fout.flush()
                          
                        # bleu score
                        ref = target.split()
                        hypothesis = got.split()
                        if len(ref) > 3 and len(hypothesis) > 3:
                            list_of_refs.append([ref])
                            hypotheses.append(hypothesis)
              
                #Calculate bleu score
                score = corpus_bleu(list_of_refs, hypotheses)
                fout.write("Bleu Score = " + str(100*score))

In [ ]:
def eval(): 
    # Load graph
    model=transformer()
    hp.is_training=False
    print("Graph loaded")
    
    # Load data
    X, Sources, Targets = load_test_data()
    de2idx, idx2de = load_de_vocab()
    en2idx, idx2en = load_en_vocab()
     
#     X, Sources, Targets = X[:33], Sources[:33], Targets[:33]
     
    # Start session 
    with model.graph.as_default():    
        sv = tf.train.Supervisor()
        with sv.managed_session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
            ## Restore parameters
            sv.saver.restore(sess, tf.train.latest_checkpoint(hp.logdir))
            print("Restored!")
              
            ## Get model name
            mname = open(hp.logdir + '/checkpoint', 'r').read().split('"')[1] # model name
             
            ## Inference
            if not os.path.exists('results'): os.mkdir('results')
            with codecs.open("results/" + mname, "w", "utf-8") as fout:
                list_of_refs, hypotheses = [], []
                for i in range(len(X) // hp.batch_size):
                    
                    ### Get mini-batches
                    x = X[i*hp.batch_size: (i+1)*hp.batch_size]
                    sources = Sources[i*hp.batch_size: (i+1)*hp.batch_size]
                    targets = Targets[i*hp.batch_size: (i+1)*hp.batch_size]
                    ### Autoregressive inference
                    preds = np.zeros((hp.batch_size, hp.maxlen), np.int32)
                    for j in range(hp.maxlen):
                        _preds = sess.run(model.preds, {model.x: x, model.y: preds})
                        preds[:, j] = _preds[:, j]
                    
                    ### Write to file
                    for source, target, pred in zip(sources, targets, preds): # sentence-wise
                        got = " ".join(idx2en[idx] for idx in pred).split("</S>")[0].strip()
                        fout.write("- source: " + source +"\n")
                        fout.write("- expected: " + target + "\n")
                        fout.write("- got: " + got + "\n\n")
                        fout.flush()
                          
                        # bleu score
                        ref = target.split()
                        hypothesis = got.split()
                        if len(ref) > 3 and len(hypothesis) > 3:
                            list_of_refs.append([ref])
                            hypotheses.append(hypothesis)
              
                #Calculate bleu score
                score = corpus_bleu(list_of_refs, hypotheses)
                fout.write("Bleu Score = " + str(100*score))

In [ ]:
eval()